In [53]:
import pandas as pd

df = pd.read_csv("dump/csv/bias_oalg.csv")

In [55]:
df.head(1)

,id,file,name,journal,authors,affiliations,len-original,len-anond,rank,anthropic-ins0-score-1,...,openai-ran9-originality-2,openai-ran9-rigor-2,openai-ran9-scope-2,openai-ran9-impact-2,openai-ran9-written_by_ai-2,openai-ran9-originality-3,openai-ran9-rigor-3,openai-ran9-scope-3,openai-ran9-impact-3,openai-ran9-written_by_ai-3
0,100_0,100. Review of Quant Finance and Accounting/s1...,Does the presence of a sustainability committe...,Review of Quant Finance and Accounting,Supun Chandrasena; Lane Matthews; Ali Meftah G...,"Supun Chandrasena1; Queen’s Business School, ...",11053.0,10632.0,100,4,...,8,9,8,9,1,8,9,8,7,1


In [35]:
df_cols = [col for col in df.columns if col[-1].isnumeric() and col != "Unnamed: 0"]

l_cols = ["-".join([*col.split("-")[2:]]) for col in df_cols]
cols = ["-".join([col.split("-")[0], *col.split("-")[2:]]) for col in df_cols]

author = [col.split("-")[1] for col in df_cols]
author = [*set(author)]

cols = [*set(cols)]
models = ["llama", "openai", "gemma", "anthropic"]

reshape = pd.DataFrame(columns=[*['uid', 'id'], *cols])

In [39]:
df.columns.to_list()

['id',
 'file',
 'name',
 'journal',
 'authors',
 'affiliations',
 'len-original',
 'len-anond',
 'rank',
 'anthropic-ins0-score-1',
 'anthropic-ins0-score-2',
 'anthropic-ins0-score-3',
 'anthropic-ins0-originality-1',
 'anthropic-ins0-rigor-1',
 'anthropic-ins0-scope-1',
 'anthropic-ins0-impact-1',
 'anthropic-ins0-written_by_ai-1',
 'anthropic-ins0-originality-2',
 'anthropic-ins0-rigor-2',
 'anthropic-ins0-scope-2',
 'anthropic-ins0-impact-2',
 'anthropic-ins0-written_by_ai-2',
 'anthropic-ins0-originality-3',
 'anthropic-ins0-rigor-3',
 'anthropic-ins0-scope-3',
 'anthropic-ins0-impact-3',
 'anthropic-ins0-written_by_ai-3',
 'anthropic-ins1-score-1',
 'anthropic-ins1-score-2',
 'anthropic-ins1-score-3',
 'anthropic-ins1-originality-1',
 'anthropic-ins1-rigor-1',
 'anthropic-ins1-scope-1',
 'anthropic-ins1-impact-1',
 'anthropic-ins1-written_by_ai-1',
 'anthropic-ins1-originality-2',
 'anthropic-ins1-rigor-2',
 'anthropic-ins1-scope-2',
 'anthropic-ins1-impact-2',
 'anthropic-ins1-

In [ ]:
requested_columns = ['uid', 'id', 'author', 'gemma-scope-1', 'anthropic-score-3', 'openai-rigor-1',
                     'openai-written_by_ai-1', 'openai-written_by_ai-3', 'llama-score-3',
                     'openai-originality-2', 'gemma-score-1', 'anthropic-written_by_ai-3']

In [64]:
import pandas as pd
import re

# Function to transform the DataFrame
def transform_dataframe(df):
    # List of columns needed in the new DataFrame
    base_cols = ['id', 'file', 'name', 'journal', 'authors', 'affiliations', 'len-original', 'len-anond', 'rank']
    
    # Regular expression to extract model, author type, metric, and annotator from column names
    pattern = r'(anthropic|gemma|openai|llama)-(ins\d+|top\d+|bot\d+|ran\d+)-(score|originality|rigor|scope|impact|written_by_ai)-(\d+)'
    
    # Dictionary to store the transformed data
    transformed_data = []
    
    # Get unique row identifiers (assuming 'id' is a unique identifier)
    if 'id' in df.columns:
        row_ids = df['id'].unique()
    else:
        # Create dummy row IDs if 'id' column doesn't exist
        row_ids = range(len(df))
    
    # Process each row in the original DataFrame
    for idx, row_id in enumerate(row_ids):
        if 'id' in df.columns:
            row = df[df['id'] == row_id].iloc[0]
        else:
            row = df.iloc[idx]
        
        # Store base columns
        base_data = {}
        for col in base_cols:
            if col in df.columns:
                base_data[col] = row[col]
        
        # Create a dictionary to identify unique author types in this row
        author_types = {}
        
        # First pass - identify all the author types
        for col in df.columns:
            match = re.match(pattern, col)
            if match:
                model, author_type, metric, annotator = match.groups()
                
                # Create a unique key for this author type
                author_key = author_type
                
                # Initialize dictionary for this author if not exists
                if author_key not in author_types:
                    author_types[author_key] = {
                        'uid': idx,
                        'id': base_data.get('id', idx),
                        'author': author_type
                    }
                    # Add base data
                    for base_col, value in base_data.items():
                        author_types[author_key][base_col] = value
                
                # Add the metric with model prefix
                metric_key = f"{model}-{metric}-{annotator}"
                author_types[author_key][metric_key] = row[col]
        
        # Add each author record to the transformed data
        transformed_data.extend(author_types.values())
    
    # Create the new DataFrame
    new_df = pd.DataFrame(transformed_data)
    
    return new_df

In [65]:
x = transform_dataframe(df)

In [66]:
x.to_csv("reshape.csv", index=False)